# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 2s 39ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,
                           y_train,
                           batch_size=32,
                           epochs=150,
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 14ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 16ms/step - loss: 0.6559 - mse: 0.6559 - val_loss: 0.2099 - val_mse: 0.2099
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2009 - mse: 0.2009 - val_loss: 0.2158 - val_mse: 0.2158
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2260 - mse: 0.2260 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1811 - mse: 0.1811 - val_loss: 0.1347 - val_mse: 0.1347
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2294 - mse: 0.2294 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1191 - mse: 0.1191 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1257 - mse: 0.1257 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1008 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0266 - mse: 0.0266 - val_loss: 0.1278 - val_mse: 0.1278
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0234 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 128/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1317 - val_mse: 0.1317
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091


[0.00908526312559843, 0.00908526312559843]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 7ms/step - loss: 0.1318 - mse: 0.1318


[0.13178403675556183, 0.13178403675556183]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28527.447201979816

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: 0.5220 - mse: 0.5220 - val_loss: 0.1772 - val_mse: 0.1772
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2653 - mse: 0.2653 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1666 - mse: 0.1666 - val_loss: 0.1402 - val_mse: 0.1402
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1869 - mse: 0.1869 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1591 - mse: 0.1591 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1737 - mse: 0.1737 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1047 - mse: 0.1047 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1346 - m

33/33 [==============================] - 0s 6ms/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1027 - val_mse: 0.1027
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0279 - mse: 0.0279 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0266 - mse: 0.0266 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0251 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1100 - val_mse: 0.1100
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102


[0.010179002769291401, 0.010179002769291401]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1107 - mse: 0.1107


[0.1106594055891037, 0.1106594055891037]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 20ms/step - loss: 0.8318 - mse: 0.8318 - val_loss: 0.1914 - val_mse: 0.1914
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2311 - mse: 0.2311 - val_loss: 0.1601 - val_mse: 0.1601
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2229 - mse: 0.2229 - val_loss: 0.1368 - val_mse: 0.1368
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1654 - mse: 0.1654 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1700 - mse: 0.1700 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1060 - mse: 0.1060 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1429 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0252 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1081 - val_mse: 0.1081
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081


[0.008055944927036762, 0.008055944927036762]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1086 - mse: 0.1086


[0.1086001992225647, 0.1086001992225647]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                      loss='mse',
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 12ms/step - loss: 0.6085 - mse: 0.6085 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1520 - mse: 0.1520 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1202 - mse: 0.1202 - val_loss: 0.0936 - val_mse: 0.0936
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1365 - mse: 0.1365 - val_loss: 0.0872 - val_mse: 0.0872
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0711 - mse: 0.0711 - val_loss: 0.0882 - val_mse: 0.0882
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0619 - mse: 0.0619 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0523 - mse: 0.0523 - val_loss: 0.0966 - val_mse: 0.0966
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0458 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0913 - val_mse: 0.0913
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.0895 - val_mse: 0.0895
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0889 - val_mse: 0.0889
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0855 - val_mse: 0.0855
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0933 - val_mse: 0.0933
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.0850 - val_mse: 0.0850
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0083 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0874 - val_mse: 0.0874
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0852 - val_mse: 0.0852
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0924 - val_mse: 0.0924
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0837 - val_mse: 0.0837
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0931 - val_mse: 0.0931
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0034 - mse: 0.0034


[0.0033713448792696, 0.0033713448792696]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0845 - mse: 0.0845


[0.08449548482894897, 0.08449548482894897]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 10ms/step - loss: 0.4833 - mse: 0.4833 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1640 - mse: 0.1640 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1086 - mse: 0.1086 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0798 - mse: 0.0798 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0527 - mse: 0.0527 - val_loss: 0.0949 - val_mse: 0.0949
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0415 - mse: 0.0415 - val_loss: 0.0971 - val_mse: 0.0971
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0436 - mse: 0.0436 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0284 - 

Epoch 63/150
33/33 [==============================] - 0s 3ms/step - loss: 6.8553e-04 - mse: 6.8553e-04 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 5.0902e-04 - mse: 5.0902e-04 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 3.7513e-04 - mse: 3.7513e-04 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 3.2775e-04 - mse: 3.2775e-04 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 2.9062e-04 - mse: 2.9062e-04 - val_loss: 0.1005 - val_mse: 0.1005
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 2.9070e-04 - mse: 2.9070e-04 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 2.6646e-04 - mse: 2.6646e-04 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 70/150
33/33 [

Epoch 124/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.0958 - val_mse: 0.0958
Epoch 125/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0154 - mse: 0.0154 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 8.7545e-04 - mse: 8.7545e-04 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 6.3077e-04 - mse: 6.3077e-04 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 131/150
33/33 [==============================] 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 8.3733e-04 - mse: 8.3733e-04


[0.0008373326854780316, 0.0008373326854780316]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0938 - mse: 0.0938


[0.0938301756978035, 0.0938301756978035]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [31]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1476 - mse: 0.1476


[0.14760762453079224, 0.14760762453079224]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [32]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

30191.584956523282

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.